**🔧 Setup Required**: Before running this notebook, please follow the [setup instructions](../../README.md#setup-instructions) to configure your environment and API keys.

# Building Document Routing with MetadataRouter

Welcome to this notebook where we'll explore the `MetadataRouter` in Haystack. This router is specifically designed to route documents based on their metadata fields, making it ideal for document filtering and categorization.

## What You'll Learn

- How to implement metadata-based document routing
- Filtering documents by category, type, or custom fields
- Building specialized processing paths for different document types
- Creating document classification pipelines

## Why MetadataRouter Matters

Different document types often need different handling:
- Filter documents by category, source, or date
- Route documents to specialized processors
- Split document collections by type
- Implement access control based on metadata

The MetadataRouter makes document classification and routing seamless and efficient.

## 1. Core MetadataRouter Components

Let's import the MetadataRouter and understand its key features:

### Key Use Cases

- Filter documents by category, source, or date
- Route documents to specialized processors
- Split document collections by type
- Implement access control based on metadata

### How It Works

The `MetadataRouter` examines document metadata and routes documents to different outputs based on rules you define. It's particularly useful in RAG pipelines where you want to process different document types differently.

In [1]:
# Import routing components
from haystack.components.routers import MetadataRouter
from haystack import Document

## 2. Understanding MetadataRouter

The `MetadataRouter` is specifically designed to route documents based on their metadata fields. Unlike the `ConditionalRouter` which can work with any data, the `MetadataRouter` is optimized for document filtering scenarios.

Let's create a practical example that routes documents by their difficulty level.

In [2]:
# Example: MetadataRouter for document filtering

# Create sample documents with various metadata
documents = [
    Document(
        content="Introduction to Haystack framework and its capabilities",
        meta={"type": "tutorial", "difficulty": "beginner", "category": "getting-started"}
    ),
    Document(
        content="Advanced RAG patterns using hybrid search",
        meta={"type": "guide", "difficulty": "advanced", "category": "rag"}
    ),
    Document(
        content="API reference for Pipeline components",
        meta={"type": "reference", "difficulty": "intermediate", "category": "api"}
    ),
    Document(
        content="Step-by-step tutorial on building your first pipeline",
        meta={"type": "tutorial", "difficulty": "beginner", "category": "pipeline"}
    ),
    Document(
        content="Best practices for production deployment",
        meta={"type": "guide", "difficulty": "advanced", "category": "deployment"}
    ),
]

# MetadataRouter uses a dictionary format with filter conditions
# Each key is the output name, and the value is a filter dict
metadata_router_rules = {
    "beginner_docs": {
        "operator": "AND",
        "conditions": [
            {"field": "difficulty", "operator": "==", "value": "beginner"}
        ]
    },
    "intermediate_docs": {
        "operator": "AND",
        "conditions": [
            {"field": "difficulty", "operator": "==", "value": "intermediate"}
        ]
    },
    "advanced_docs": {
        "operator": "AND",
        "conditions": [
            {"field": "difficulty", "operator": "==", "value": "advanced"}
        ]
    },
}

# Create the MetadataRouter
doc_router = MetadataRouter(rules=metadata_router_rules)

print("Testing MetadataRouter:\n")
print("=" * 80)

for doc in documents:
    result = doc_router.run(documents=[doc])
    
    # Find which output the document was routed to
    routed_to = None
    for output_name, output_docs in result.items():
        if output_docs and len(output_docs) > 0:
            routed_to = output_name
            break
    
    print(f"\nDocument: '{doc.content[:50]}...'")
    print(f"Metadata: {doc.meta}")
    print(f"Routed to: {routed_to if routed_to else 'No route'}")
    print("-" * 80)

Testing MetadataRouter:


Document: 'Introduction to Haystack framework and its capabil...'
Metadata: {'type': 'tutorial', 'difficulty': 'beginner', 'category': 'getting-started'}
Routed to: beginner_docs
--------------------------------------------------------------------------------

Document: 'Advanced RAG patterns using hybrid search...'
Metadata: {'type': 'guide', 'difficulty': 'advanced', 'category': 'rag'}
Routed to: advanced_docs
--------------------------------------------------------------------------------

Document: 'API reference for Pipeline components...'
Metadata: {'type': 'reference', 'difficulty': 'intermediate', 'category': 'api'}
Routed to: intermediate_docs
--------------------------------------------------------------------------------

Document: 'Step-by-step tutorial on building your first pipel...'
Metadata: {'type': 'tutorial', 'difficulty': 'beginner', 'category': 'pipeline'}
Routed to: beginner_docs
-----------------------------------------------------------

## 3. Building a Complete Metadata Routing Pipeline

Now let's build a complete pipeline that uses the MetadataRouter to process documents differently based on their difficulty level. We'll route documents to separate document stores.

In [3]:
# Build a pipeline with metadata routing and visualization
from haystack import Pipeline
from haystack.components.writers import DocumentWriter
from haystack.document_stores.in_memory import InMemoryDocumentStore

# Create a pipeline for metadata-based document routing
metadata_pipeline = Pipeline()
doc_router = MetadataRouter(rules=metadata_router_rules)

# Create the metadata router with the same rules
metadata_pipeline.add_component("metadata_router", doc_router)

# Add separate document stores for different difficulty levels
beginner_store = InMemoryDocumentStore()
intermediate_store = InMemoryDocumentStore()
advanced_store = InMemoryDocumentStore()

beginner_writer = DocumentWriter(document_store=beginner_store)
intermediate_writer = DocumentWriter(document_store=intermediate_store)
advanced_writer = DocumentWriter(document_store=advanced_store)

metadata_pipeline.add_component("beginner_writer", beginner_writer)
metadata_pipeline.add_component("intermediate_writer", intermediate_writer)
metadata_pipeline.add_component("advanced_writer", advanced_writer)

# Connect the router outputs to respective writers
metadata_pipeline.connect("metadata_router.beginner_docs", "beginner_writer.documents")
metadata_pipeline.connect("metadata_router.intermediate_docs", "intermediate_writer.documents")
metadata_pipeline.connect("metadata_router.advanced_docs", "advanced_writer.documents")

🚅 Components
  - metadata_router: MetadataRouter
  - beginner_writer: DocumentWriter
  - intermediate_writer: DocumentWriter
  - advanced_writer: DocumentWriter
🛤️ Connections
  - metadata_router.beginner_docs -> beginner_writer.documents (list[Document])
  - metadata_router.intermediate_docs -> intermediate_writer.documents (list[Document])
  - metadata_router.advanced_docs -> advanced_writer.documents (list[Document])

In [4]:
# Test the pipeline with our documents
print("\n\nRunning pipeline with sample documents...")
result = metadata_pipeline.run({"metadata_router": {"documents": documents}})

# Show the results
print(f"\n\nDocuments routed:")
print(f"  - Beginner: {beginner_store.count_documents()} documents")
print(f"  - Intermediate: {intermediate_store.count_documents()} documents")
print(f"  - Advanced: {advanced_store.count_documents()} documents")



Running pipeline with sample documents...


Documents routed:
  - Beginner: 2 documents
  - Intermediate: 1 documents
  - Advanced: 2 documents


## 4. Advanced Metadata Routing Patterns

### Complex Filter Conditions

You can create more sophisticated routing rules with multiple conditions and operators:

```python
complex_rules = {
    "technical_tutorials": {
        "operator": "AND",
        "conditions": [
            {"field": "type", "operator": "==", "value": "tutorial"},
            {"field": "category", "operator": "==", "value": "technical"}
        ]
    },
    "recent_guides": {
        "operator": "AND",
        "conditions": [
            {"field": "type", "operator": "==", "value": "guide"},
            {"field": "year", "operator": ">=", "value": 2024}
        ]
    }
}
```

### Production Use Cases

1. **Access Control**: Route documents based on security level or permissions
2. **Quality Filtering**: Separate high-quality from low-quality content
3. **Time-Based Routing**: Route recent vs. historical documents differently
4. **Source-Based Processing**: Handle documents from different sources with specialized logic
5. **Multi-Tenant Systems**: Route documents by tenant or organization

## 5. Summary and Best Practices

### What We Covered

- ✅ Document filtering with MetadataRouter
- ✅ Building complete routing pipelines with document stores
- ✅ Creating sophisticated filter conditions
- ✅ Visualizing metadata-based routing flows

### Key Takeaways

1. **Document-Specific**: MetadataRouter is optimized for document routing scenarios
2. **Filter Conditions**: Use operator and conditions to define precise routing rules
3. **Multiple Outputs**: Route documents to different paths based on metadata
4. **Production Ready**: Efficient filtering for large document collections

### Best Practices

- Define clear metadata schemas for your documents
- Test routing rules with diverse document samples
- Consider using default/fallback routes for unmatched documents
- Monitor routing distribution to ensure balance
- Use consistent metadata field names across your system

### Supported Operators

- `==`: Equal to
- `!=`: Not equal to
- `>`: Greater than
- `>=`: Greater than or equal to
- `<`: Less than
- `<=`: Less than or equal to
- `in`: Value in list
- `not in`: Value not in list

### Next Steps

- Implement multi-condition routing rules
- Add metadata enrichment before routing
- Build category-specific processing pipelines
- Integrate with document classification models
- Create monitoring dashboards for routing metrics

Happy document routing! 📚🚀